In [18]:
import os

r3_path = os.getenv('WASMR3_PATH', '/home/don/wasm-r3')

def assert_cpus_disabled(start, end):
    with open('/sys/devices/system/cpu/online', 'r') as f:
        online_cpus = f.read().strip()
        for cpu in range(start, end+1):
            assert str(cpu) not in online_cpus, f"CPU {cpu} is enabled"
def check_cpu_governor(start, end):
    for cpu in range(start, end+1):
        governor_file = f"/sys/devices/system/cpu/cpu{cpu}/cpufreq/scaling_governor"
        if os.path.exists(governor_file):
            with open(governor_file, 'r') as f:
                governor = f.read().strip()
                assert governor == 'performance', f"CPU {cpu} governor is not set to performance"
        else:
            print(f"CPU {cpu} does not exist or does not have a scaling governor")
def assert_cover_all(expected_dirs):
    online_tests_path = os.path.join(r3_path, 'tests/online')
    actual_dirs = [name for name in os.listdir(online_tests_path) if os.path.isdir(os.path.join(online_tests_path, name))]
    try:
        assert set(actual_dirs) == set(expected_dirs)
    except AssertionError:
        missing_dirs = set(expected_dirs) - set(actual_dirs)
        extra_dirs = set(actual_dirs) - set(expected_dirs)
        print(f"Assertion failed: Missing directories: {missing_dirs}, Extra directories: {extra_dirs}")
        raise

# run ~/cpu.sh
check_cpu_governor(0, 15)
assert_cpus_disabled(16, 31)

# Setup evaluation suite

yusung_set = [
    "bullet",
    "factorial",
    "ffmpeg",
    "fractals",
    "funky-kart",
    "game-of-life",
    "gotemplate",
    "hnset-bench",
    'hydro',
    "jqkungfu",
    "lichess",
    "mandelbrot",
    "ogv",
    "onnxjs",
    "pacalc",
    'parquet',
    "playnox",
    "roslyn",
    "rustpython",
    "sandspiel",
    "sqlgui",
    "sqlpractice",
    "takahirox",
    "tic-tac-toe", # flaky
    "timestretch",
    "vaporboy",
    "video",
    "waforth",
    "wasmsh",
    "wheel",
]

jakob_set = [
    "boa",
    "commanderkeen",
    "ffmpeg",
    "fib",
    "figma-startpage",
    "funky-kart",
    "game-of-life",
    "guiicons",
    'image-convolute',
    "jsc",
    'multiplyDouble',
    "multiplyInt",
    "ogv",
    "pathfinding",
    "riconpacker",
    "rtexviewer",
    "sandspiel",
    "sqlgui",
    "takahirox",
    "video",
]

doehyun_set = [
    'livesplit',
    'rfxgen',
    'rguilayout',
    'rguistyler',
    'rtexpacker',
    'vim-wasm',
]

# These are excluded as they don't appear in either Made with WebAssembly(https://madewithwebassembly.com/) or Awesome-Wasm(https://github.com/mbasso/awesome-wasm)
excluded_set = [
    "handy-tools",
    'heatmap',
    "kittygame",
    'visual6502remix',
    'noisereduction',
    'skeletal',
    'uarm',
    'virtualkc',
]

print('yusung_set: ', len(yusung_set))
print('jakob_set: ', len(jakob_set))
print('doehyun_set: ', len(doehyun_set))

union = list(set(yusung_set) | set(jakob_set) | set(doehyun_set))
intersection = list(set(yusung_set) & set(jakob_set))
print('union: ', len(union))
print('exclude: ', len(excluded_set))
assert_cover_all(union + excluded_set)

testset = union
metrics = {testname: { 'trace_match': {}, 'record_metrics': {}, 'replay_metrics': {}} for testname in testset }

yusung_set:  30
jakob_set:  20
doehyun_set:  6
union:  48
exclude:  8


In [13]:
import subprocess
import json

# Trace difference experiment
frontends = ['wasabi']
timeout = 120

def run_command(testname, frontend):
    command = f". ~/.bashrc && timeout {timeout}s npm test -- -t {testname}"
    result = subprocess.run(command, shell=True, capture_output=True, text=True)
    isNormal = result.returncode == 0
    if not isNormal:
        print(result.args)
        print(result.stderr)
    return [testname, frontend, isNormal]

results = [run_command(testname, frontend, i) for i, testname in enumerate(testset) for frontend in frontends]
for testname, frontend, isNormal in results:
    metrics[testname]['trace_match'][frontend] = isNormal

with open('metrics.json', 'w') as f:
    json.dump(metrics, f, indent=4)

. ~/.bashrc && timeout 120s npm test --  -t takahirox -p 8081

. ~/.bashrc && timeout 120s npm test --  -t onnxjs -p 8082

. ~/.bashrc && timeout 120s npm test --  -t rustpython -p 8083

. ~/.bashrc && timeout 120s npm test --  -t waforth -p 8084
thread 'main' panicked at crates/replay_gen/src/wasmgen.rs:449:5:
Failed to execute wasm-merge first time: Output { status: ExitStatus(unix_wait_status(256)), stdout: "", stderr: "[parse exception: control flow inputs are not supported yet (at 0:2009)]Fatal: error in parsing wasm input: index.wasm\n" }
note: run with `RUST_BACKTRACE=1` environment variable to display a backtrace

. ~/.bashrc && timeout 120s npm test --  -t gotemplate -p 8085

. ~/.bashrc && timeout 120s npm test --  -t lichess -p 8087

. ~/.bashrc && timeout 120s npm test --  -t timestretch -p 8094

. ~/.bashrc && timeout 120s npm test --  -t vim-wasm -p 8095

. ~/.bashrc && timeout 120s npm test --  -t fractals -p 8097

. ~/.bashrc && timeout 120s npm test --  -t wheel -p 810

In [17]:
import subprocess, time, re, json, os

# Record overhead experiment

with open('metrics.json', 'r') as f:
    metrics = json.load(f)

def extract_samples_and_mean(output):
    match = re.search(r"recorded (\d+) samples, mean = ([\d\.]+)", output)
    samples = int(match.group(1))
    mean = float(match.group(2))
    return [samples, mean]

def extract_cycle_counts(output):
    pattern = r"(\d+(?:,\d+)*)\s+cpu_core/cpu-cycles/"
    matches = re.findall(pattern, output)
    cycle_counts = [int(match.replace(',', '')) for match in matches]
    return cycle_counts

test_input = """
DevTools listening on ws://127.0.0.1:9966/devtools/browser/f72191be-fbd6-4fbd-b21f-2703612f1f13
 Performance counter stats for 'CPU(s) 0-15':
    40,125,664,880      cpu_core/cpu-cycles/                                                  
       6.157604349 seconds time elapsed
 Performance counter stats for 'CPU(s) 0-15':
     2,702,581,574      cpu_core/cpu-cycles/                                                  
       0.267278301 seconds time elapsed
"""
test_input_2 = """
================
Run online tests
================
WARNING: You need a working internet connection
WARNING: Tests depend on third party websites. If those websites changed since this testsuite was created, it might not work
fib  -Histogram: V8.ExecuteMicroSeconds recorded 581 samples, mean = 9993.9 (flags = 0x41)

581 9993.9
nvm
"""

assert extract_cycle_counts(test_input) == [40125664880, 2702581574]
assert extract_samples_and_mean(test_input_2) == [581, 9993.9]

timeout = 120 # seconds
chromium_path = os.getenv('WASMR3_PATH', '/home/don/.cache/ms-playwright/chromium-1105/chrome-linux/chrome')
CDP_PORT = os.getenv('CDP_PORT', 8080)

def run_command(testname, option):
    try:
        subprocess.run(["killall", "-9", "chrome"])
        chromium_cmd = f". ~/.bashrc && {chromium_path} --renderer-process-limit=1 --no-sandbox --remote-debugging-port={CDP_PORT} --js-flags='--slow-histograms' --renderer-cmd-prefix='bash ./perf.sh'"
        wasmr3_cmd = f". ~/.bashrc && timeout {timeout}s npm test -- --evalRecord {option} -t {testname}"
        with open('output.txt', 'w') as f: subprocess.Popen(chromium_cmd, shell=True, stdout=f , stderr=f)
        result = subprocess.run(wasmr3_cmd, shell=True, stdout=subprocess.PIPE, text=True)
        time.sleep(3)
        with open('output.txt', 'r') as f: output = f.read()
        cycle_counts = extract_cycle_counts(output)
        samples, mean = extract_samples_and_mean(result.stdout)
        return [testname, option, cycle_counts, samples, mean]
    except Exception as e:
        print(f"Failed to run {testname} with {option}, error: {e}")
        return [testname, option, -1, -1, -1]
    
results = []
for testname in metrics:
    isNormal = metrics[testname]['trace_match']['wasabi']
    if isNormal:
        for option in ['--noRecord', '']:
            testname, _, cycles, samples, mean = run_command(testname, option) 
            key = 'original' if option == '' else 'instrumented'
            metrics[testname]['record_metrics'][key] = {'samples': samples, 'mean': mean, 'cycles': cycles}

with open('metrics.json', 'w') as f:
    json.dump(metrics, f, indent=4)

In [35]:
import subprocess, json

# Replay characteristic experiment

with open('metrics.json', 'r') as f:
    metrics = json.load(f)

timeout = 120 # seconds
engine_kind = ['sm', 'sm-base', 'sm-opt', 'v8', 'v8-liftoff', 'v8-turbofan', 'jsc', 'jsc-int','jsc-bbq','jsc-omg', 'wizeng','wizeng-int','wizeng-jit','wizeng-dyn','wasmtime','wasmer','wasmer-base']
wizard_engine_kind = ['wizeng','wizeng-int','wizeng-jit','wizeng-dyn']
opt_kind = ['noopt', 'split', 'merge', 'custom', 'benchmark'] # custom and benchmark are technically not replay opt though
parallel = False

def get_replay_wasm(testname, opt):
    regex = ''
    match opt:
        case 'noopt':
            regex = 'merge|split|custom|benchmark'
        case 'split':
            regex = 'noopt|merge|custom|benchmark'
        case 'merge':
            regex = 'noopt|split|custom|benchmark'
        case 'custom':
            regex = 'noopt|split|merge|benchmark'
        case 'benchmark':
            regex = 'noopt|split|merge|custom'
        case _:
            exit('invalid op')
    find_command = f"find {r3_path}/tests/online/{testname} -name replay.wasm | grep -vE '{regex}'"
    find_result = subprocess.run(find_command, shell=True, capture_output=True, text=True)
    replay_path = find_result.stdout.strip()
    return replay_path

def run_wish_you_were_fast(testname, engine, opt):
    global metrics
    replay_path = get_replay_wasm(testname, opt)
    command = f". ~/.bashrc && RUNS=1 ENGINES={engine} timeout {timeout}s compare-engines.bash {replay_path}"
    result = subprocess.run(command, shell=True, capture_output=True, text=True)
    isNormal = 'o' if result.returncode == 0  else ''
    if not isNormal:
        print(result.args)
        print(result.stderr)
        metrics[testname]['replay_metrics'][engine][opt] = {}
    else:
        runtime = float(result.stdout.split(":")[-1].strip())
        metrics[testname]['replay_metrics'][engine][opt] |= { 'runtime': runtime }

def run_wizard(testname, engine, opt):
    global metrics
    replay_path = get_replay_wasm(testname, opt)
    command = f". ~/.bashrc && timeout {timeout}s  wizeng.x86-64-linux --metrics --monitors=profile {replay_path}"
    result = subprocess.run(command, shell=True, capture_output=True, text=True)
    isNormal = result.returncode == 0
    if not isNormal:
        print(result.args)
        print(result.stderr)
        metrics[testname]['replay_metrics'][engine][opt] = {}
    else:
        monitor, profile = result.stdout.split("pregen:time_us")
        profile = 'pregen:time_us' + profile
        # Make replay_metrics after "pregen:time_us" a key of some object
        metrics[testname]['replay_metrics'][engine][opt] |= {line.rsplit(":", 1)[0].strip(): line.rsplit(":", 1)[1].strip().replace("μs", "").strip() for line in profile.split("\n") if line}

for testname in metrics:
    isNormal = metrics[testname]['trace_match']['wasabi']
    if isNormal:
        for engine in engine_kind:
            metrics[testname]['replay_metrics'][engine] = {}
            for opt in opt_kind:
                metrics[testname]['replay_metrics'][engine][opt] = {}
        for engine in engine_kind:
            for opt in ['benchmark']:
                run_wish_you_were_fast(testname, engine, opt)
        # for engine in wizard_engine_kind:
        for engine in ['wizeng-int']:
            for opt in opt_kind:
                run_wizard(testname, engine, opt)

with open('metrics.json', 'w') as f:
    json.dump(metrics, f, indent=4)

. ~/.bashrc && timeout 60s  wizeng.x86-64-linux --metrics --monitors=profile 

. ~/.bashrc && RUNS=1 ENGINES=wizeng-jit timeout 60s compare-engines.bash /home/don/wasm-r3/tests/online/mandelbrot/benchmark/bin_0/replay.wasm

. ~/.bashrc && RUNS=1 ENGINES=wizeng-dyn timeout 60s compare-engines.bash /home/don/wasm-r3/tests/online/mandelbrot/benchmark/bin_0/replay.wasm

. ~/.bashrc && timeout 60s  wizeng.x86-64-linux --metrics --monitors=profile 

. ~/.bashrc && timeout 60s  wizeng.x86-64-linux --metrics --monitors=profile 

. ~/.bashrc && RUNS=1 ENGINES=wizeng-jit timeout 60s compare-engines.bash /home/don/wasm-r3/tests/online/rfxgen/benchmark/bin_0/replay.wasm
!NullCheckException
	in X86_64Assembler.emit_rex_bb_r_m() [/home/don/titzer/virgil/lib//asm/x86-64/X86_64Assembler.v3 @ 2095:38]
	in X86_64Assembler.movss_m_s() [/home/don/titzer/virgil/lib//asm/x86-64/X86_64Assembler.v3 @ 877:32]
	in X86_64MacroAssembler.emit_mov_m_r() [src/engine/x86-64/X86_64MacroAssembler.v3 @ 260:45]
	in Macro